In [0]:

%pip install requests
import requests
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("FetchStackOverflowAPIData").getOrCreate()

#Find the trending tags
api_url = "https://api.stackexchange.com/2.3/tags?order=desc&sort=popular&site=stackoverflow&fromdate=1669814400&todate=1691452800"
response = requests.get(api_url)
data = response.json()
trending_tags = data['items']

# Transforming the data
df = spark.createDataFrame(trending_tags)

# Storing data on Delta Lake
table_location = "/delta/stack_overflow_tags"
df.write.format("delta").mode("overwrite").save(table_location)

# Create Delta table
spark.sql("drop table if exists stack_overflow_tags")
spark.sql(f"CREATE TABLE stack_overflow_tags USING DELTA LOCATION '{table_location}'")

# Query for insights
top_trending_tags = spark.sql("SELECT * FROM stack_overflow_tags LIMIT 10")
print("stack_overflow_popular_tags")
top_trending_tags.show()

#Comments with at least a score of 10 on Ask Ubuntu
api_url = "https://api.stackexchange.com/2.3/comments?order=desc&sort=votes&site=askubuntu&min=10"
response = requests.get(api_url)
data = response.json()
comments = data['items']

# Transforming the data
df = spark.createDataFrame(comments)

# Storing data on Delta Lake
table_location = "/delta/ask_ubuntu_comments"
df.write.format("delta").mode("overwrite").save(table_location)

# Create Delta table

spark.sql("drop table if exists ask_ubuntu_comments")
spark.sql(f"CREATE TABLE ask_ubuntu_comments USING DELTA LOCATION '{table_location}'")

# Query - Comments with Score >= 10
print("ask_ubuntu_comments")
high_score_comments = spark.sql("SELECT * FROM ask_ubuntu_comments WHERE score >= 10")
high_score_comments.show()
